# Things we own database

Postgress database to keep inventory of all of the things that we own in preparation for moving.

Schema design saved on [DBDesigner](https://app.dbdesigner.net/designer/schema/401855).

In [1]:
# imports
import psycopg2
import pandas as pd
from contextlib import contextmanager

**Variables**

Declaring databse variables to allow for connection.

In [2]:
# Variables
database = "House"
user = "iainmcl"
password = "empire"
host = "127.0.0.1"
port = "5433"

## Context manager

Create a context manager to handle connection to the postgres database and ensure that it is closed.

In [3]:
# Database context manger
@contextmanager
def connect(database=database, user=user, password=password, host=host, port=port):
    conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
    try:
        yield conn
    finally: 
        conn.close()

## Setup functions

- Create tables
- Execute script

In [10]:
def execute_script(cur, script):
    with open(script) as s:
        query = s.read()
        cur.execute(query)

In [11]:
def create_tables(sql_path="./scripts/HouseMoving_postgres_create.sql"):
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, sql_path)
        conn.commit()

## Select functions

- Select all


In [12]:
def select_all(table_name="Items", outfile=False):
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, "./scripts/select_all.sql")
        data = cur.fetchall()
    print(data)        

In [13]:
def drop_all():
    with connect() as conn:
        cur = conn.cursor()
        execute_script(cur, "./scripts/drop_all.sql")
        conn.commit()

In [14]:
try:
    drop_all()
except Exception as e:
    print("Table already dropped.")
    print("\t-", e)

In [22]:
create_tables()

InvalidForeignKey: there is no unique constraint matching given keys for referenced table "House"


## Export functions